In [1]:
######### Selenium으로 
######### naver 크롤링 연습(1): 검색 후 뉴스 첫 페이지의 텍스트 가져오기
######### 학습용으로 크롤링 한 것이나 문제가 될 시 바로 삭제하겠습니다


# imports
import time
import pickle
import pandas as pd

# 셀레니움 imports
import re
import requests
from selenium import webdriver
# content = h.find_element_by_css_selector('dl > dd:nth-child(1)').text li:nth-child(3) 이건 안되고 elements[1]은 됨? 연구 ㄱ

#날짜 분리하기 with Regular Expression
def date_refine(str_date):
    try:
        # etc)2020.00.00 포맷
        date = '\d{4}.(\d{2}).(\d{2}).'
        dt = re.compile(date)
        if dt.search(str_date) is not None:
            return dt.search(str_date).group(0)

        # etc)몇시간, 몇 분전 포맷  
        hours = '\d{1,2}\w{1,2}\s\w'
        hr = re.compile(hours)
        if hr.search(str_date) is not None:
            return hr.search(str_date).group(0)    

    except Exception as e:
        print("시간 명시 안되어있네요")
        print(e)
        pass

# pickle 저장
def save_to_pickle(df,search_title):
    pickle.dump(df, open('./data/{}_df.pkl'.format(search_title),'wb'))
    print('저장완료')

def get_all_news_elements(page):
     # 첫 페이지 크롤링
    df = pd.DataFrame(columns=['title', 'author','date', 'content'])

    # page 수만큼 loop
    for i in range(page): 
        lists = driver.find_elements_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > ul.type01 > li')
        for lst in lists:
            try: 
                news_date = date_refine(lst.find_element_by_css_selector('dl > dd.txt_inline').text)
                # 데이터프레임에 추가하기
                df.loc[len(df)] = {
                    # 긁어모아 df에 저장하기
                    'title':lst.find_element_by_css_selector('dl > dt > a').get_attribute("title"),
                    'author':lst.find_element_by_css_selector('dl > dd.txt_inline > span._sp_each_source').text,
                    'date': news_date,
                    'content':lst.find_elements_by_css_selector('dl > dd')[1].text
                }   
            except Exception as e:
                print('텍스트 가져오기 에러')
                print(e)
                pass

        # 페이지 다 돌았으면 다음 페이지 가기전에 stop
        if i == (page-1):
            break
    
        # 다음 페이지
        driver.find_element_by_css_selector('div.paging > a.next').click() 
    
    print("끝","꼴")
    
    return df
    
    # driver.find_elements_by_class_name('_sp_each_url._sp_each_title').get_attribute("title")
def crawl(search,page):
    time.sleep(2) # 잠시 기다리기
    # div가 여러개일 경우 div.classname으로 명시해줄것
    
    # 검색창 클릭
    driver.find_element_by_css_selector('#nx_search_form > fieldset > div.greenbox > span > input').click()
    # 검색창 xpath : driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/div.greenbox/span/input').click() 
    # 검색어 입력
    driver.find_element_by_css_selector('#nx_query').send_keys(search)
    # 검색 버튼 클릭
    driver.find_element_by_css_selector('#nx_search_form > fieldset > button').click() 
    # 최신순
    driver.find_elements_by_css_selector('.news_option > ul.sort > li')[1].click()
    time.sleep(1)
    
    
    # df 받기 잠시만!!
    news_df = get_all_news_elements(page) 
    # pickle 파일이름
    s_title = '네이버_' + search
    # 저장
    save_to_pickle(news_df,s_title)
    print("파일이 저장되었습니다. 파일 이름:", s_title+'_df.pkl')
    # excel, csv 저장
    path ='C://Users//user//Desktop//savefile_'
    #news_df.to_excel(path+s_title+'.xlsx',sheet_name='sheet1')
    #news_df.to_csv(path+s_title+'.csv', sep=',', na_rep='NaN', encoding="euc-kr")
def news():
    print('네이버 뉴스 검색 크롤링 연습입니다. 알맞는 형태의 파일을 올려주세요')
    
    search = input('검색어를 설정해주세요:  ')
    page = int(input('검색 할 페이지 수를 설정해주세요:  '))
    #start = input('시작날짜를 입력해주세요 [ex)2020.07.22]:  ')
    #end = input('종료할 날짜를 입력해주세요 [ex)2020.07.22]:  ')
    #search = '물병'
    crawl(search,page)
    
    
# Chrome driver 부르기
driver = webdriver.Chrome('c:/cdriver/chromedriver.exe')
driver.implicitly_wait(3) # 
url = 'https://search.naver.com/search.naver?where=news/'
driver.get(url)

# sleep: 프로세스 자체를 지연 (무조건 지연) 
# implicitly_wait : 파싱 시간을 기다려주는 메소드
# (wait 해주지 않으면, 브라우저에서 바로 파싱이 되지 않으면 에러 후 종료가 됨)
# (즉, 로딩 시간을 기다려 주지 않는 것)


In [2]:
# 크롤링 시작
news()

네이버 뉴스 검색 크롤링 연습입니다. 알맞는 형태의 파일을 올려주세요
검색어를 설정해주세요:  물병
검색 할 페이지 수를 설정해주세요:  3
끝 꼴
저장완료
파일이 저장되었습니다. 파일 이름: 네이버_물병_df.pkl


In [3]:
df = pickle.load(open('./data/네이버_물병_df.pkl','rb'))


In [4]:
df

,title,author,date,content
0,"외출이 즐거워지는 칼 오스카, 스텐빨대물병 출시",글로벌에듀,56분 전,스타일리쉬 하고 미니멀한 디자인과 러블리한 색상이 특징인 스웨덴 브랜드 칼 오스카(...
1,"겟잇뷰티2020, 황승언의 '브이라인 메이커' 비결은?",미디어리퍼블릭,1시간 전,"여러 루틴 중 그녀가 V라인 관리를 위해 마시는 '기분전환 브리라인티'는 ""스케줄 ..."
2,[★별자리운세] 2020년7월25일 (토요일) 동서양 별자리 타로운세,중도일보,2시간 전,*금전운★ ★물병자리 [1.20~2.18] [LOVE] 서로의 상상만으로 오해를 하...
3,"횡성군, 소상공인 일반음식점 스테인리스 물병 지원",쿠키뉴스,2시간 전,강원 횡성군은 소상공인 일반음식점 775개소를 선정해 음식점에서 소독 가능한 위생물...
4,"마운틴, 로드, 하이브리드…내게 맞는 자전거는?",미주중앙일보,2시간 전,cannondale.com ■액서서리 및 구매 요령 필수 액서서리로는 헬멧과 바이크...
5,"횡성군, 소상공인 일반음식점 스테인리스 물병 지원",업코리아,4시간 전,"업코리아, UPKOREA ▲ 횡성군, 소상공인 일반음식점 스테인리스 물병 지원 [업..."
6,"월성원전 맥스터 증설, 경주시민 81.4% 찬성",뉴시스,4시간 전,시민단체 등은 김 위원장이 밖으로 나오자 격렬히 반발하며 차량에 탑승하지 못하게 막...
7,"[창원 용호동 맛집] '딜리스' 정보승 대표, ""남녀노소 즐기고 맛 보실 수 있는 ...",업코리아,5시간 전,가게외관 내부 접시 그릇 물병 화장실 모든것 하나하나 너무 신경많이 쓴 게 느껴진다...
8,"횡성군, 소상공인 일반음식점 스테인리스 물병 지원",뉴스타운,5시간 전,횡성군은 소상공인 일반음식점 775개소를 선정하여 음식점에서 소독 가능한 위생물품(...
9,[금요일 별자리 운세] 보란 듯이 해피엔딩,이코노믹리뷰,6시간 전,"물병자리 : 정신 차려, 정신 차려… 사람 사이의 문제 때문에 조금 고민을 하게 되..."
